In [1]:
import pymarc as pymarc
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
#from fuzzywuzzy import process

In [16]:
hidden_count = 1
output_count = 1
eta = 0.6
u = np.array([0.92, 0.62, 1, 0])
label = 1
tau_1 = 0.5
tau_2 = 0.7

z = np.random.randn(len(u[range(3)]), hidden_count)
u[range(3)]

samples = np.array([[0.12, 0.02, 1, 0], [0.92, 0.88, 1, 1]])
for u in samples:
    print(u)

[ 0.12  0.02  1.    0.  ]
[ 0.92  0.88  1.    1.  ]


In [1]:
class NeuralNet:
    
    def __init__(self, input=3, hidden=3, output=1):
        self.input = input
        self.hidden = hidden
        self.output = output
        
    def create_nodes(self):
        pass
            
        

class Node:
    
    def __init__(self):
        return None
          
    def _get_weights(self):
        return self.__weights
    
    def _set_weights(self, value):
        self.__weights = value
    
    weights = property(_get_weights, _set_weights)
   
    @property
    def bias(self):
        return self.__bias
    
class Layer:
    
    node_list = []
    
    def __init__(self, node_count):        
        self.node_count = node_count
        self.nodes = [Node() for node in range(self.node_count)]
    
    

    
    

        
        
def aggregate(values, weights, bias):
    return np.dot(values, weights) + bias * 1

def activate(aggregation):
    return 1 / (1 + np.exp(-aggregation))

def compute_total_error(target, output):
    return np.sum(1/2 * (target - output)**2)

def compute_e_partial_o1(target, output):
    return 2 * 1/2 * (target - output) * -1

def compute_o1_partial_net(output):
    return output * (1 - output)

In [ ]:
X = np.array([
    [-2,4,-1],
    [4,1,-1],
    [1, 6, -1],
    [2, 4, -1],
    [6, 2, -1],

])
weights = np.zeros(len(X[0]))
weights

In [ ]:
x = np.zeros((3, 8))
x.shape = (6, 4)
x

In [ ]:
for input in range(3):
    for hidden in range(3):
        print(input, "-->", hidden)

#### Input values
`input = np.array([0.05, 0.1])`

#### Input weights + bias
```input_weights_1 = np.array([0.15, 0.2])
input_weights_2 = np.array([0.25, 0.3])
bias_1 = np.array([0.35])```

#### Hidden layer weights + bias
```
hidden_weights_1 = np.array([.4, .45])
hidden_weights_2 = np.array([.5, .55])
bias_2 = np.array([.6])


h1 = Node(input, input_weights_1, bias_1, aggregate, activate)
h2 = Node(input, input_weights_2, bias_1, aggregate, activate)

h_out = np.concatenate([h1.activate(), h2.activate()])

o1 = Node(h_out, hidden_weights_1, bias_2, aggregate, activate)
o2 = Node(h_out, hidden_weights_2, bias_2, aggregate, activate)

o_out = np.concatenate([o1.activate(), o2.activate()])

e_o1 = compute_total_error(0.01, o1.activate())
e_o2 = compute_total_error(0.99, o2.activate())

e_total = e_o1 + e_o2

e_partial_o1 = compute_e_partial_o1(0.01, o1.activate())
o1_partial_net = compute_o1_partial_net(o1.activate())
o1_partial_w5 = 1 * h1.activate()
w5 = 0.4 - .5 * e_partial_o1 * o1_partial_net * o1_partial_w5

0.4 - 0.5 * 0.082167041
w5
```

In [2]:
#sample = np.array([[["Florian van Roekel", "Roekel, Florian van, 1980- photographer."], ["How Terry likes his coffee", "How Terry likes his coffee : a photo odyssey into office life /"]], [["Sun Dog Propaganda", "Sun Dog Propaganda,"], ["Banned in DC", "Banned in DC : photos and anecdotes from the DC punk underground (79-85) /"]]])
#for s in sample:
#    z = [fuzz.ratio(x[0], x[1])*0.01 for x in s]
#    print(z)

# Predict
def predict(s):
    prediction = []

    for p in s:
        if p >= 0.7:
            prediction.append(1)
        if p <= 0.5:
            prediction.append(0)
        if (p < 0.7) & (p > 0.5):
            prediction.append("?")
    return prediction

#np.random.seed(1)

hidden_count = 2
output_count = 1

sample = np.array([0.18, 0.08])
label = 0
tau_1 = 0.5
tau_2 = 0.7

samples = np.array([[0.38, 0.58, 1], [0.97, 0.98, 1]])

#bias = np.ones(hidden_count)
#labels = np.array([[0], [1]])





In [3]:
# Feed forward
def activate(aggregation):
    return 1 / (1 + np.exp(-aggregation))

def error_tau_1(s):
    return np.sum(s**2)

def error_tau_2(s):
    return np.sum((1 - s)**2)

# s
def dQ_wrt_a_o(output_layer_activation):    
    return 2 * output_layer_activation
    #return 2 * (output_layer_activation - 1)

# s
def da_out_wrt_a_i(output_layer_activation):
    return output_layer_activation * (1 - output_layer_activation)

# y
def di_out_wrt_w(output_layer_input):
    return output_layer_input

# input --> hidden
#b_h = np.zeros(hidden_count)
hidden_layer_weights = np.random.randn(len(sample), hidden_count)

# hidden --> out
#b_out = np.zeros(output_count)
output_layer_weights = np.random.randn(hidden_count, output_count)

for i in range(1500):
    hidden_layer_input = np.dot(sample, hidden_layer_weights)
    hidden_layer_activation = activate(hidden_layer_input)

    output_layer_input = np.dot(hidden_layer_activation, output_layer_weights)
    output_layer_activation = activate(output_layer_input)



    #def error_tau_2(s, t):
    #    return np.sum((1 - s[t == 1])**2)

    #if label == 0:
    total_error = error_tau_1(output_layer_activation) + error_tau_2(output_layer_activation)
    print("Error: ", label - total_error)
    print("--")
    print("Output: ", output_layer_activation)
    print("--")
    print("Prediction: ", predict(output_layer_activation))
    # Partial derivatives wrt output layer weights


    d1 = dQ_wrt_a_o(output_layer_activation)
    d2 = da_out_wrt_a_i(output_layer_activation)
    d3 = di_out_wrt_w(output_layer_input)

    dw = (d1 * d2 * d3)

    # Partial derivatives wrt hidden layer weights
    # s

    d4 = dQ_wrt_a_o(output_layer_activation)
    d5 = da_out_wrt_a_i(output_layer_activation)
    d6 = output_layer_weights
    d7 = output_layer_input * (1 - output_layer_input) # y(1 - y)
    d8 = sample

    dz = (d4 * d5 * d6 * d7 * d8)

    output_layer_weights -= (0.4 * dw.T)
    
    hidden_layer_weights -= (0.4 * dz.T)
    



Error:  -0.654525682832
--
Output:  [ 0.22203806]
--
Prediction:  [0]
Error:  -0.646785910599
--
Output:  [ 0.22908866]
--
Prediction:  [0]
Error:  -0.638975448623
--
Output:  [ 0.23639476]
--
Prediction:  [0]
Error:  -0.631121638839
--
Output:  [ 0.24395153]
--
Prediction:  [0]
Error:  -0.62325503886
--
Output:  [ 0.25175109]
--
Prediction:  [0]
Error:  -0.615409201291
--
Output:  [ 0.25978218]
--
Prediction:  [0]
Error:  -0.607620305244
--
Output:  [ 0.26802985]
--
Prediction:  [0]
Error:  -0.599926628022
--
Output:  [ 0.27647525]
--
Prediction:  [0]
Error:  -0.592367853959
--
Output:  [ 0.28509554]
--
Prediction:  [0]
Error:  -0.584984229428
--
Output:  [ 0.29386384]
--
Prediction:  [0]
Error:  -0.577815587169
--
Output:  [ 0.30274941]
--
Prediction:  [0]
Error:  -0.57090027821
--
Output:  [ 0.31171793]
--
Prediction:  [0]
Error:  -0.564274063962
--
Output:  [ 0.32073195]
--
Prediction:  [0]
Error:  -0.557969032241
--
Output:  [ 0.3297516]
--
Prediction:  [0]
Error:  -0.55201260698


Output:  [ 0.49999769]
--
Prediction:  [0]
Error:  -0.500000000009
--
Output:  [ 0.49999791]
--
Prediction:  [0]
Error:  -0.500000000007
--
Output:  [ 0.49999811]
--
Prediction:  [0]
Error:  -0.500000000006
--
Output:  [ 0.49999829]
--
Prediction:  [0]
Error:  -0.500000000005
--
Output:  [ 0.49999845]
--
Prediction:  [0]
Error:  -0.500000000004
--
Output:  [ 0.4999986]
--
Prediction:  [0]
Error:  -0.500000000003
--
Output:  [ 0.49999873]
--
Prediction:  [0]
Error:  -0.500000000003
--
Output:  [ 0.49999885]
--
Prediction:  [0]
Error:  -0.500000000002
--
Output:  [ 0.49999896]
--
Prediction:  [0]
Error:  -0.500000000002
--
Output:  [ 0.49999906]
--
Prediction:  [0]
Error:  -0.500000000001
--
Output:  [ 0.49999915]
--
Prediction:  [0]
Error:  -0.500000000001
--
Output:  [ 0.49999923]
--
Prediction:  [0]
Error:  -0.500000000001
--
Output:  [ 0.4999993]
--
Prediction:  [0]
Error:  -0.500000000001
--
Output:  [ 0.49999937]
--
Prediction:  [0]
Error:  -0.500000000001
--
Output:  [ 0.49999943]

--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  

--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  

Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0

Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0.5]
--
Prediction:  [0]
Error:  -0.5
--
Output:  [ 0

In [ ]:
for i in range(1500):
       
    # step 1: net input of hidden layer
    # [n_samples, n_features] dot [n_features, n_hidden]
    # -> [n_samples, n_hidden]
    z_h = np.dot(samples, w_h)

    def activate(aggregation):
        return 1 / (1 + np.exp(-aggregation))

    # step 2: activation of hidden layer
    a_h = activate(z_h)

    # step 3: net input of output layer
    # [n_samples, n_hidden] dot [n_hidden, n_classlabels]
    # -> [n_samples, n_classlabels]
    z_out = np.dot(a_h, w_out) #+ b_out

    # step 4: activation output layer
    a_out = activate(z_out)

    
    print("---")
    print("Prediction: ", predict(a_out))

    # Cost 
    def error_tau_1(s, t):
        return np.sum(s[t == 0]**2)

    def error_tau_2(s, t):
        return np.sum((1 - s[t == 1])**2)

    Q = error_tau_1(a_out, labels) + error_tau_2(a_out, labels)

    def derive_Q_tau_1(s, t):
        return 2 * s[t == 0]

    def derive_Q_tau_2(s, t):
        return 2 * (1 - s[t == 1])

    def derive_sigma(s):
        return s * (1 - s)

    def derive_o_in(h):
        return h

    dQ_1 = derive_Q_tau_1(a_out, labels)
    dQ_2 = derive_Q_tau_2(a_out, labels)

    print("a_out: ", a_out)
    
    dw_out = (dQ_1 + dQ_2) * derive_sigma(a_out) * derive_o_in(z_out)

    dw_h = (dQ_1 + dQ_2) * derive_sigma(a_out) * w_out * derive_sigma(z_out) * samples

    w_out -= (0.05 * dw_out)

    w_h -= (0.05 * dw_h.T)

  

In [ ]:

                ##################
                # Backpropagation
                ##################

                # [n_samples, n_classlabels]
                sigma_out = a_out - y_train_enc[batch_idx]

                # [n_samples, n_hidden]
                sigmoid_derivative_h = a_h * (1. - a_h)

                # [n_samples, n_classlabels] dot [n_classlabels, n_hidden]
                # -> [n_samples, n_hidden]
                sigma_h = (np.dot(sigma_out, self.w_out.T) *
                           sigmoid_derivative_h)

                # [n_features, n_samples] dot [n_samples, n_hidden]
                # -> [n_features, n_hidden]
                grad_w_h = np.dot(X_train[batch_idx].T, sigma_h)
                grad_b_h = np.sum(sigma_h, axis=0)

                # [n_hidden, n_samples] dot [n_samples, n_classlabels]
                # -> [n_hidden, n_classlabels]
                grad_w_out = np.dot(a_h.T, sigma_out)
                grad_b_out = np.sum(sigma_out, axis=0)

                # Regularization and weight updates
                delta_w_h = (grad_w_h + self.l2*self.w_h)
                delta_b_h = grad_b_h # bias is not regularized
                self.w_h -= self.eta * delta_w_h
                self.b_h -= self.eta * delta_b_h

                delta_w_out = (grad_w_out + self.l2*self.w_out)
                delta_b_out = grad_b_out  # bias is not regularized
                self.w_out -= self.eta * delta_w_out
                self.b_out -= self.eta * delta_b_out

In [ ]:
# Activation of hidden layer
def activate(aggregation):
    return 1 / (1 + np.exp(-aggregation))
h_a = activate(h_in)

# Net input of output layer
# [n_samples, n_hidden] dot [n_hidden, n_classlabels]
# -> [n_samples, n_classlabels]
o_in = np.dot(h_a, output_weights)
o_a = activate(o_in)

# Cost function
def error_t1(s, p):
    p = p.reshape((2, 1))
    return np.sum(s[p == 0]**2)
        
def error_t2(s, p):
    p = p.reshape((2, 1))
    return np.sum((1 - s[p == 1])**2)

    
Q = np.sum(o_a**2) + np.sum((1 - o_a)**2)

error_t1(o_a, labels) + error_t2(o_a, labels)

# Predict
def predict(s):
    predictions = []    
    if s[s > 0.7]:
        predictions.append(1)
    if s[s < 0.5]:
        predictions.append(0)
    if s[(s < 0.7) & (s > 0.5)]:
        predictions.append("?")
    return predictions

# Backpropagation
# dQ/dw = dQ/ds * ds/dl * dl/dw
def derive_Q_tau_1(s, p):

    return 2 * s[p == 0]

def derive_Q_tau_2(s, p):

    return 2 * (1 - s[p == 1])

#derive_Q_tau_1(o_a, labels) * derive_Q_tau_2(o_a, labels)
def derive_sigma(s):
    return s * (1 - s)

def derive_o_in(h_a):
    return h_a

#dw = derive_Q_tau_1(o_a, labels) * derive_Q_tau_2(o_a, labels) * derive_sigma(o_a) * derive_o_in(h_a)

h_in

In [ ]:
#o_a[labels == 0].T
#np.dot(o_a[labels == 0], output_weights.T)
hid = np.random.randn(hidden_count, len(samples[0]))
out = np.random.randn(hidden_count, len(samples[0])) 

p = labels.reshape((2, 1))
o_a

In [6]:
(1 - 0.02)**2
#0.02**2

0.9603999999999999

In [52]:
df = pd.read_csv("/home/tat2/Desktop/original-rows/report2.csv")
#df[df["source_name_1"] == "Bani Abidi"]
training = df[:925]
testing = df[925:]
#testing
#copy = training[training["source_name_1"].notnull() & training["target_creator"].notnull()].copy()
copy = df.copy()

#df['race_label'] = df.apply (lambda row: label_race (row),axis=1)

def compute_title(row):
    return .01 * fuzz.ratio(row["source_title"], row["target_title"])

def compute_pub(row):
    if row["source_pub"].notnull() and row["target_pub"].notnull():        
        return .01 * fuzz.ratio(row["source_pub"], row["target_pub"])

#df['salary'] = df['SAL-RATE'].apply(money_to_float)

#def calculate_area(row):
#    return row['height'] * row['width']

#rectangles_df.apply(calculate_area, axis=1)

copy["cmp_title"] = copy.apply(compute_title, axis=1)
#copy["cmp_pub"] = copy.apply(compute_pub, axis=1)


In [54]:
copy.to_csv("/home/tat2/Desktop/original-rows/titles.csv")

In [40]:
copy

,source_id,target_id,source_name_1,source_name_2,source_title,source_pub,target_creator,target_contributor,target_title,target_pub,cmp_title
12,69f663c4c510ce52937dbb5c1b6eafb6,070a91df51b8e933f2d666f7c99eee19,"Charara, Tarek",NaN,A l'Ombre de Chatila,Etincelles,"Charara, Tarek.",NaN,A l'ombre de Chatila /,"Editions Étincelles,",0.01
13,69f663c4c510ce52937dbb5c1b6eafb6,7e0b566755c6330346a4858aede21691,"Charara, Tarek",NaN,A l'Ombre de Chatila,Etincelles,"Charara, Tarek, 1961- ...",NaN,À l'ombre de Chatila /,"Éd. Étincelles,",0.01
14,3f717d4a1aa993008e5abb5014b34501,1db9d19307cab6e197248ea70b5b2197,Cynthia Connolly,NaN,Lengths and Breaths,Water Row Press,"Ranaldo, Lee.",NaN,Lengths and breaths /,"Water Row Press,",0.01
15,58c9a27f7aaf6f23fbf9e5a718f54e04,13df09aa39a0bbf0d9c16f8d7928c3d5,Maxwell Anderson,NaN,See You Soon (またあうひまで),Bemojake,"Anderson, Maxwell, 1986-",NaN,See you soon =,"Bemojake,",0.01
16,58c9a27f7aaf6f23fbf9e5a718f54e04,dd578cfb8ed43eb156c3cc392251489f,Maxwell Anderson,NaN,See You Soon (またあうひまで),Bemojake,"Anderson, Maxwell, 1986-",NaN,See you soon /,"Bemojake,",0.01
17,8b792fc3fabd2957ce533ae575d26042,b9e0e3da12cc8768b61f58ed17d078bf,David Schoerner,NaN,Photographs,Hassla Books,"Schoerner, David.",NaN,Photographs /,"Hassla Books,",0.01
18,b0ed44e0b7adcc152a57a97e88d126b8,ee6577be1f18bedf5279ec5d7d6f645f,Kate Steciw,NaN,The Strangeness of This Idea,Hassla Books,"Steciw, Kate.",NaN,The strangeness of this idea /,"Hassla Books,",0.02
19,f1bcfe2f0d917c8c9e47e4dd872d8666,de587c2e9effef4ab0d8b45fb822ac4a,Anne Collier,NaN,Woman With A Camera (35mm),Hassla Books,"Collier, Anne, 1970-","McDonough, Tom,",Woman with a camera (35mm) /,"Hassla Books,",0.02
20,f1bcfe2f0d917c8c9e47e4dd872d8666,41c6b099c0f9873cc62dc76376d8de54,Anne Collier,NaN,Woman With A Camera (35mm),Hassla Books,"Collier, Anne",NaN,Woman with a camera (35 mm),Artpace,0.02
21,b17fe44cb9e57f9e3e46aa86d24a204a,dfe3c4d33b8d4356b9d0455c7f3e4868,Sam Falls,NaN,Color Dying Light,Hassla Books,"Falls, Sam.",NaN,Color dying light /,"Hassla Books,",0.01


In [4]:
with open("iPL-results2-rev2.mrc", "rb") as fh:
    recs = pymarc.MARCReader(fh)
    for i, rec in enumerate(recs):                 
        print(rec["245"]["a"])

Weight and body measurement changes In college freshmen /
Discrete Symmetries and CP Violation :
Dynamical electro-weak symmetry breaking from deformed AdS: vector mesons and effective couplings
[Do our legislators deserve a pay rise?]
10 :
Inspiration.
Inspiration.
Banned in DC :
Banned in DC :
Banned in DC :
Banned in DC :
Banned in DC :
Banned in DC :
Banned in DC :
Banned in DC :
Banned in DC
Box of ice boxes :
A l'ombre de Chatila /
À l'ombre de Chatila /
Lengths and breaths /
Smooth soul.
Shalom Sesame.
De quien es el portaligas? /
Through a Blue Lens /
Grammont sélection.
¿De quien es el portaligas? /
Through a blue lens /
Symposium issue :
Joe Franklin :
Grammont sélection.
Asymmetric catalysis on industrial scale :
Adolescentes en el siglo XXI :
Relaciones caribeñas :
Aspectos de la climatología y de los rendimientos de maíz y soja en zonas productoras mundiales :
Vicisitudes negro africanas en iberoamérica :
Circulación :
Circulación :
Circulación :
Patrones de circ

The fifteen biggest lies about the economy :
Dans l'empire des sens =
The Oxford book of Japanese short stories /
Dans l'empire des sens =
Geisha /
The Artie Shaw story.
Harrison Ford /
The fifty-seventh annual Midwest Clinic, 2003 :
Look where God has brought us /
Intégrale Claude Nougaro
Ô Toulouse. :
English songs /
Angiogenesis :
The reader /
Cambridge Handbook of Psychology, Health and Medicine /
Beautiful creatures /
Encyclopedia of African American history, 1896 to the present :
Beautiful creatures /
Prophecy in the book of Jeremiah /
The power of optical/IR interferometry :
Nakba :
Beyond Babylon :
F Ampersand /
Learning absences, 1986 /
Hamrā Hajuraāmā =
Hamra hajurama =
The secretary of dreams.
Photodebut :
Neighbourhood /
John Maclean :
John MacLean :
John Maclean :
Two and two /
Two and two :
City /
John MacLean :
John MacLean :
John MacLean :
John MacLean :
City /
John MacLean :
John MacLean :
The Oxford handbook of Byzantine studies /
Oxford handbook of engineering a

Whip it /
Derby /
Industria, virtus, et fortitudo /
The jazz standards :
Videonale. 13 :
Bright wings :
The complete Columbia album collection /
Self-portrait without a bicycle /
100 great poems for girls /
The beauty chorus /
Adventures in Oz.
The ultimate rock pop fake book :
The Brady bunch.
Social studies :
Social studies :
Social studies :
Social studies :
The Norton anthology of Latino literature /
Reid's read-alouds 2 :
I was Indian :
Global soccer mom :
Merrill Shea studio recital.
Suicidal Tendencies :
Amazon Kindle #3
Amazon Kindle #2
Amazon Kindle #1
Global soccer mom :
Con-form :
Reid's read-alouds 2 :
Blood on the stage, 1950-1975 :
100 journeys for the spirit :
The tsarina's slippers =
The drama collection :
100 Best British Classics.
Happy to you /
Happy to you
Happy to you
Estaría bien poner un título aquí /
Estaría bien poner un título aquí
Sunnydell farm /
Interior nature /
Cityscapes + birdmen /
Jacquie Maria Wessels :
Cityscapes + birdmen
Cityscapes + birdmen 

Spider-sense Spider-Man :
Mastering type :
Advances in computer science, engineering & applications :
Microsoft Word 2010 bible /
Half life :
Half life :
Bearing witness (to AIDS) /
Bearing witness (to AIDS) /
Bearing witness.
Bearing witness (to AIDS) /
Olifantenpaadjes =
Olifantenpaadjes / Desire lines. 3rd. ed.
Olifantenpaadjes =
Olifantenpaadjes = Desire lines
Olifantenpaadjes =
Olifantenpaadjes =
Olifantenpaadjes =
Political aesthetics /
Hart Crane's poetry :
Controversial bodies :
The best women's travel writing 2010 :
Environmental history of the Hudson River :
The Ottoman world /
Angels & ducats :
Sound intentions :
Grave concerns /
The best women's travel writing 2010 :
Nekopashi :
Showa 88 /
Showa 88.
Bravo - Black hits vol. 23
Bravo black hits
Bravo black hits
Bravo - Black hits vol. 23 :
Lounge anthology
Design and Optimization of an Austenitic TRIP Steel for Blast and Fragment Protection.
Strawberry snow :
Bouchon Bakery /
The Back in the Day Bakery cookbook :
A history of

The mermaid of Brooklyn /
The mermaid of Brooklyn :
The mermaid of Brooklyn :
The mermaid of Brooklyn
Life in blue /
Evžen Sobek :
Life in Blue /
Life in Blue.
Enkeltje Schiphol =
The nuclear enterprise :
Nuclear energy encyclopedia :
Publishing the fine and applied arts, 1500-2000 /
London :
Reading, writing, and reasoning :
A country merchant, 1495-1520 :
The tempest /
Christopher Columbus and the Age of Exploration for kids with 21 activities /
What happened to Sophie Wilder :
The nuclear enterprise :
Harry Potter and the Deathly Hallows.
Ready player one /
Best kept secret /
Ready player one /
Harry Potter and the prisoner of Azkaban /
The butler :
The bartender's tale /
The sins of the father /
Harry Potter and the Chamber of Secrets /
Ready player one /
They came to Baghdad /
100 acoustic lessons /
Acoustic guitar licks goldmine.
100 rock lessons /
The essential guide to Asperger's syndrome /
200 rock licks.
100 jazz lessons /
100 blues lessons /
100 country lessons /
200 countr

A Falling Horizon /
Micro :
Casino Jack /
A wish and a prayer :
An introduction to sustainable transportation :
The best of Soul train /
Tough as nails :
Joanne Fluke Christmas bundle /
That evening sun /
Droid 2 :
Power, national security, and transformational global events :
The care of time /
North Korea demystified /
After Artest :
Avoiding common nursing errors /
If only I were-- /
If only I were-- =
The rogue agent /
The last girl /
Harbinger /
North Korea demystified /
Celebrity /
Celebrity /
City Administration in the Ancient Near East :
Rescue me.
The amazing Spider-Man.
Rescue me.
Rescue me.
City administration in the ancient Near East :
Ahmedabad 600 :
Recovering the self :
Sociology and human rights :
City administration in the ancient Near East :
Martin Parr's best books of the decade /
Good God! :
Dirty negative /
Empty land, promised land, forbidden land /
On the other side of the mountains /
Empty land, promised land, forbidden land :
The Sochi project /
The Sochi proje

Mao III
Butterfly had a dream /
Cho no mita yume =
Yuri Gagarin :
Yuri Gagarin :
Midwest dirt
Trying to find the ocean /
Love letter /
Down and out in the South [exhibition] /
Down and out in the south.
Down and out in the south /
5 Seconds of Summer :
5 Seconds of Summer.
Anonymity /
5 Seconds of Summer.
How did we end up here?
Sounds good feels good
Smaakmakers :
Collected poems /
The raven and the falcon :
Manuscript found in Accra :
Manuscript found in Accra :
The literary heritage of the Arabs :
The blue sapphire of the mind :
How to find your spirit animal :
Life in the second circle :
Feminist biblical interpretation :
The wisdom of menopause :
The dealer is the devil :
More great ideas a day ... :
What I Believe.
The 20th century.
100 Local Heroes.
Literature :
Long way home :
Astonishing the Wild Pigs :
Ask a science teacher, how everyday stuff really works :
The pathway of my life 1966-1986 /
Rough sets and current trends in computing :
Rough sets and current trends in comput

Proceedings of Coastal Dynamics 2009 :
Connected :
Connected :
Transactions of the Kansas Academy of Science.
38 :
Proceedings of Coastal Dynamics 2009
活断層における地電位差観測(1)-北陸観測所坑内およひ&#x3099;牛首断層における観測-
Visible library /
Black nature :
How beautiful it is and how easily it can be broken :
Use trouble :
150 cartoon classics.
Masterplots II.
Of this world :
Haunted hills :
"Yippee ki-yay moviegoer" :
American old time fiddle tunes :
The real book.
Jets from young stars :
Encyclopedia of social problems /
The essential guide to business etiquette /
Sumurun :
Pow wow :
New urbanism and beyond :
What is medicine? :
Growing cooler :
Transit Cafe /
Knowledge science, engineering and management :
Childhood, well-being and a therapeutic ethos /
Ask me about my divorce :
Human chain /
On tact, & the made up world /
The poem I turn to :
Women in dialogues :
The best American poetry 2009 /
Kicking in :
Ibsen and the modern self /
Paris in the Middle Ages /
Johnnie To Kei-Fung's PTU /
Mafia movies :
Th